In [1]:
!pip install nemo_toolkit[asr] 

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.9/806.9 kB 22.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install datasets transformers librosa jiwer git-lfs accelerate torch num2words soundfile tqdm audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 2.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 1.9 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 7.7 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.0
    Uninstalling scipy-1.14.0:
      Successfully uninstalled scipy-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which

In [3]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("AILAB-VNUHCM/vivos", trust_remote_code=True)

# Split the dataset into train, validation, and test sets (80-10-10)
def split_dataset(dataset, train_size=0.8, val_size=0.1):
    # Split the dataset into train and remaining (val+test)
    train_test_split_data = dataset['train'].train_test_split(test_size=1 - train_size)
    train_dataset = train_test_split_data['train']
    remaining_dataset = train_test_split_data['test']

    # Split the remaining dataset into validation and test
    val_test_split_data = remaining_dataset.train_test_split(test_size=val_size / (1 - train_size))
    val_dataset = val_test_split_data['train']
    test_dataset = val_test_split_data['test']

    # Create a DatasetDict
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })
    
    return dataset_dict

# Split the dataset
dataset = split_dataset(dataset)

Generating train split:   0%|          | 0/11660 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/760 [00:00<?, ? examples/s]

In [4]:
import pandas as pd

# Paths to the CSV files
csv_file1 = '/kaggle/input/tele-vi/annoted/metadata.csv'
csv_file2 = '/kaggle/input/tele-vi/notannoted/metadata.csv'

# Read both CSV files
data1 = pd.read_csv(csv_file1)
data2 = pd.read_csv(csv_file2)

# Concatenate the dataframes
merged_data = pd.concat([data1, data2], ignore_index=True)

# Save the merged dataframe to a new CSV file
merged_data.to_csv('/kaggle/working/metadata.csv', index=False)

print("Merged CSV file saved successfully.")


Merged CSV file saved successfully.


In [5]:
import pandas as pd
import os
import json
import tqdm

# Paths and directories
csv_file_path = '/kaggle/working/metadata.csv'  # Adjust this path accordingly
output_base_dir = '/kaggle/working/'

# Read the CSV file
data = pd.read_csv(csv_file_path)

# Split dataset into train, validation, test (80-10-20 split)
train_split = data.sample(frac=0.8, random_state=42)
remaining_data = data.drop(train_split.index)
val_split = remaining_data.sample(frac=0.1, random_state=42)
test_split = remaining_data.drop(val_split.index)

# Define splits and corresponding file names
splits = {
    'train': train_split,
    'validation': val_split,
    'test': test_split
}

# Loop through each split and save JSON manifest
for split_name, split_data in splits.items():
    json_file = os.path.join(output_base_dir, f'{split_name}_manifest.json')

    # Create and open the JSON manifest file
    with open(json_file, 'w', encoding='utf8') as jf:
        # Loop through dataset split and write metadata to JSON
        for index, row in tqdm.tqdm(split_data.iterrows(), total=len(split_data)):
            path = row['file']
            text = row["text"]
            duration = row["duration"]
            
            # Write metadata to JSON
            manifest_item = {
                "audio_filepath": path,  
                "duration": duration,
                "text": text
            }
            json.dump(manifest_item, jf, ensure_ascii=False)
            jf.write('\n')  # Write newline character after each JSON object

print("Dataset split and manifest files created successfully.")

100%|██████████| 2159/2159 [00:00<00:00, 8715.92it/s]

Dataset split and manifest files created successfully.


In [4]:
import pandas as pd
import os
import json
import tqdm
import librosa  # Thư viện để load file âm thanh
import soundfile as sf  # Thư viện để lưu file âm thanh
import numpy as np
import random
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# Paths and directories
csv_file_path = '/kaggle/working/metadata.csv'  # Adjust this path accordingly
output_base_dir = '/kaggle/working/'

# Read the CSV file
data = pd.read_csv(csv_file_path)

# Split dataset into train, validation, test (80-10-20 split)
train_split = data.sample(frac=0.8, random_state=42)
remaining_data = data.drop(train_split.index)
val_split = remaining_data.sample(frac=0.1, random_state=42)
test_split = remaining_data.drop(val_split.index)

# Define splits and corresponding file names
splits = {
    'train': train_split,
    'validation': val_split,
    'test': test_split
}

# Define audio augmentation pipeline
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.1),
    TimeStretch(min_rate=0.8, max_rate=1.1, p=0.1),
    PitchShift(min_semitones=-3, max_semitones=3, p=0.1),
    Shift(min_shift=-0.3, max_shift=0.3, p=0.1),
])

# Function to augment audio
def augment_audio(file_path):
    # Load the audio file
    samples, sample_rate = librosa.load(file_path, sr=None)
    
    # Apply augmentation
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    
    return augmented_samples, sample_rate

# Loop through each split and save JSON manifest
for split_name, split_data in splits.items():
    json_file = os.path.join(output_base_dir, f'{split_name}_manifest.json')

    # Create and open the JSON manifest file
    with open(json_file, 'w', encoding='utf8') as jf:
        # Loop through dataset split and write metadata to JSON
        for index, row in tqdm.tqdm(split_data.iterrows(), total=len(split_data)):
            path = row['file']
            text = row["text"]
            duration = row["duration"]
            
            # Augment audio
            augmented_audio, sample_rate = augment_audio(path)
            
            # Optional: Save augmented audio to new file (if needed)
            augmented_file_path = os.path.join(output_base_dir, f"augmented_{os.path.basename(path)}")
            sf.write(augmented_file_path, augmented_audio, sample_rate)

            # Write metadata to JSON
            manifest_item = {
                "audio_filepath": augmented_file_path,  # Use augmented file path
                "duration": duration,
                "text": text
            }
            json.dump(manifest_item, jf, ensure_ascii=False)
            jf.write('\n')  # Write newline character after each JSON object

print("Dataset split, augmentation, and manifest files created successfully.")


 91%|█████████ | 1969/2159 [00:40<00:03, 50.78it/s]/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
100%|██████████| 2159/2159 [00:43<00:00, 49.27it/s]

Dataset split, augmentation, and manifest files created successfully.


In [6]:
import pandas as pd
import textwrap

def create_transcript_from_csv(csv_file, output_file):
    all_transcripts = []

    # Read the CSV file and collect the text data
    data = pd.read_csv(csv_file)
    for text in data['text']:
        if isinstance(text, str):
            all_transcripts.append(text)
        elif pd.isna(text):
            continue
        else:
            all_transcripts.append(str(text))

    # Join all text segments with a space
    combined_transcript = " ".join(all_transcripts)

    # Split the combined transcript into lines of no more than 40 characters
    wrapped_lines = textwrap.wrap(combined_transcript, width=40)

    # Write the lines to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        for line in wrapped_lines:
            f.write(line + "\n")

# Define the path to the CSV file
csv_file_path = '/kaggle/working/metadata.csv'

# Call the function to create the transcript file
create_transcript_from_csv(csv_file_path, "transcript_all.json")

print("Transcript file created successfully.")


Transcript file created successfully.


In [7]:
import os

# Create the directory
os.makedirs('/kaggle/working/Fast-Conformer/hparam', exist_ok=True)
os.makedirs('/kaggle/working/Fast-Conformer/dict_N', exist_ok=True)

print("Created successfully.")


Created successfully.


In [7]:
!wget https://raw.githubusercontent.com/ThienKym123/Fast-Conformer/main/process_asr_text_tokenizer.py -O /kaggle/working/Fast-Conformer/process_asr_text_tokenizer.py
!wget https://raw.githubusercontent.com/ThienKym123/Fast-Conformer/main/hparam/fast-conformer_ctc_bpe.yaml -O /kaggle/working/Fast-Conformer/hparam/fast-conformer_ctc_bpe.yaml
!wget https://raw.githubusercontent.com/ThienKym123/Fast-Conformer/main/speech_to_text_ctc_bpe.py -O /kaggle/working/Fast-Conformer/speech_to_text_ctc_bpe.py

--2024-09-09 07:57:28--  https://raw.githubusercontent.com/ThienKym123/Fast-Conformer/main/process_asr_text_tokenizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9132 (8.9K) [text/plain]
Saving to: '/kaggle/working/Fast-Conformer/process_asr_text_tokenizer.py'

/kaggle/working/Fas 100%[===================>]   8.92K  --.-KB/s    in 0s      

2024-09-09 07:57:29 (38.9 MB/s) - '/kaggle/working/Fast-Conformer/process_asr_text_tokenizer.py' saved [9132/9132]

--2024-09-09 07:57:30--  https://raw.githubusercontent.com/ThienKym123/Fast-Conformer/main/hparam/fast-conformer_ctc_bpe.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubuse

In [8]:
!python /kaggle/working/Fast-Conformer/process_asr_text_tokenizer.py\
        --data_file="/kaggle/working/transcript_all.json" \
        --data_root="/kaggle/working/Fast-Conformer/dict_N" \
        --vocab_size=1024 \
        --tokenizer="spe" \
        --no_lower_case \
        --spe_type="bpe" \
        --spe_character_coverage=1.0 \
        --log


[NeMo I 2024-09-09 07:57:42 sentencepiece_tokenizer:316] Processing /kaggle/working/transcript_all.json and store at /kaggle/working/Fast-Conformer/dict_N/tokenizer_spe_bpe_v1024
sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/kaggle/working/transcript_all.json --model_prefix=/kaggle/working/Fast-Conformer/dict_N/tokenizer_spe_bpe_v1024/tokenizer --vocab_size=1024 --shuffle_input_sentence=true --hard_vocab_limit=false --model_type=bpe --character_coverage=1.0 --bos_id=-1 --eos_id=-1
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/transcript_all.json
  input_format: 
  model_prefix: /kaggle/working/Fast-Conformer/dict_N/tokenizer_spe_bpe_v1024/tokenizer
  model_type: BPE
  vocab_size: 1024
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iteration

In [9]:
!wandb login edfa6bb2b01b5b2e74dd9f112d7ba0da853ebd0e

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!HYDRA_FULL_ERROR=1 python /kaggle/working/Fast-Conformer/speech_to_text_ctc_bpe.py --config-path="/kaggle/working/Fast-Conformer/hparam"

[NeMo W 2024-09-09 07:58:00 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo I 2024-09-09 07:58:00 speech_to_text_ctc_bpe:12] Hydra config: name: FastConformer-CTC-BPE
    init_from_pretrained_model: stt_en_fastconformer_ctc_large
    model:
      sample_rate: 16000
      log_prediction: true
      ctc_reduction: mean_volume
      skip_nan_grad: false
      train_ds:
        manifest_filepath: /kaggle/working/train_manifest.json
        sample_rate: ${model.sample_rate}
        batch_size: 32
        shuffle: true
        num_workers: 8
        pin_memory: true
        max_duration: 20
        min_duration: 0.1
        is_tarred: false
        tarred_audio_filepaths: null
        shuffle_n: 2048
        bucketing_

In [ ]:
import nemo
from nemo.core import ModelPT

# Đường dẫn đến file .nemo
nemo_file = '/kaggle/input/fast-conformer_ctc_bpt_tele-vi/other/model/1/model.nemo'

# Load mô hình từ file .nemo
model = ModelPT.restore_from(restore_path=nemo_file)


In [ ]:
import soundfile as sf
from IPython.display import Audio

# Đường dẫn đến file âm thanh
audio_file = '/kaggle/input/tele-vi/annoted/wavs/1.wav'

# Đọc file âm thanh
audio_data, sample_rate = sf.read(audio_file)

# Tiến hành nhận dạng tiếng nói (assuming `model` is your ASR model)
transcription = model.transcribe([str(audio_file)])

print(transcription)

Audio(data=audio_data, rate=sample_rate)

In [ ]:
model.eval()

In [ ]:
import json
from nemo.collections.asr.metrics.wer import word_error_rate

# Define the path to your test manifest file
test_manifest = "/kaggle/working/test_manifest.json"

# Read data from the manifest file
with open(test_manifest, 'r', encoding='utf-8') as f:
    manifest_data = [json.loads(line.strip()) for line in f]

predictions = []
references = []

# Loop through each sample in the manifest and perform predictions
for sample in manifest_data:
    audio_file = sample['audio_filepath']
    transcript = sample['text']

    # Perform inference with each audio sample
    transcription = model.transcribe([audio_file])[0]

    # Add the prediction and the reference to the respective lists
    predictions.append(transcription)
    references.append(transcript)

    # Print the result after each sample
    print(f"Reference: {transcript}")
    print(f"Prediction: {transcription}")
    print("------")


In [ ]:

# Calculate accuracy based on the number of correct sentences
correct_count = sum(1 for pred, ref in zip(predictions, references) if pred == ref)
total_count = len(predictions)
accuracy = correct_count / total_count

print(f"\nAccuracy: {accuracy * 100:.2f}%")